# Using capellambse with Jinja2

Welcome to the py-capellambse jinja2 templating showcase. This notebook will
introduce you into writing jinja2 templates where you'll plant model
information and diagrams using jinja2's amazing templating language. With Jinja2
you are able to generate any text-based format(HTML, XML, CSV, LaTex,...) but
during this tutorial we will only generate .html files. The jinja2 syntax is
inspired by python. Check out their [docs]! 

[docs]: https://jinja2docs.readthedocs.io/en/stable/

Below code loads the needed libraries and instantiates a test model:

In [7]:
import jinja2
import capellambse

import pathlib


path_to_model = "../tests/data/melodymodel/5_0/Melody Model Test.aird"
model = capellambse.MelodyModel(pathlib.Path(path_to_model))

FileNotFoundError: [Errno 2] No such file or directory: '../tests/data/melodymodel/5_0/Melody%20Model%20Test.afm'

In the following we want to make a template to document all modelled actors from
the logical layer. Therefore we define a template string where we iterate over
all actors and plant the name, uuid and description into it.

*Hint: Make sure that you know of capella's [metamodel] as we are implementing the
capellambse.layers as close as possible to it while being as efficiently and pythonic
we can be currently. This knowledge can shorten used statements immensely!*

[metamodel]: https://dsd-dbs.github.io/py-capellambse/start/intro-to-api.html

In [ ]:
env = jinja2.Environment()
templ = """
<h1>Actor definitions</h1>
{% for actor in model.la.all_components.by_is_actor(True) %}
    <h2>{{ actor.name }}</h2>
    <h3>Actor definition</h3>
    <p>UUID: {{ actor.uuid }}</p>
    <p>{{ actor.description }}</p>
{% endfor %}
"""
env.from_string(templ).render(model=model)

As an extra: We don't use the UUID from capella in our documents. If you still
need an identifier, in the following it's explained how we are doing it:

With the capella PVMT, one of the various capella [addons], you can make property
value groups and then set arbitrary values with these. capellambse is able to recognize
the pvmt extension and gives read and write access. In our workflows we are maintaining
an ID database for all model elements. If that is done you can access pvmt attributes
like:

´´´html
{{ ... }}
<p>ID: {{ actor.pvmt["Group.Identification.MY MODEL ID"] }}</p>
{{ ... }}
´´´

For a PVMT showcase look into [TODO: pvmt-showcase notebook].

[addons]: https://www.eclipse.org/capella/addons.html

## Filters and object manipulation

Now to step up our templating-game, we'll bring in more complexity.
We want a template that documents functional context of all actors. For that
iff the actor has a non-empty functions attribute we make a table with columns
for function's uid, name, description and `FunctionalExchange`s.

We can define variables in the template via the set statement. Furthermore
the builtin jinja [filters] already give much power for object manipulation during
rendering. Here we use map(.) to create `FunctionalExchange` iterators and sum these
up into one large list that stores all `FunctionalExchange`s that have an an actor
as either source or target. In the table for-loop we then filter on this lookup
container and set outgoing and incoming `FunctionalExchange`s that we need for
the last column.

The jupyter environment is great for writing templates b/c you can investigate
possible attributes of objects right away in another cell.

*Hint: You can define custom filter-functions and add them to the Environment.filters.*

[filters]: https://jinja.palletsprojects.com/en/3.0.x/templates/#builtin-filters

In [ ]:
templ1 = """
<h1>Actor definitions</h1>
{% set fexs = model.la.actor_exchanges | map(attribute="func_exchanges") | map('list')
    | sum(start=[]) | list %}
{% for actor in model.la.all_actors %}
    <h2>{{ actor.name }}</h2>
    <h3>Actor definition</h3>
    <p>{{ actor.description }}</p>
    <h3>Actor functions</h3>
    {% if actor.functions %}
        <p>The table below identifies functions of {{ actor.name }}.</p>
        <table>
            <thead>
                <tr>
                    <th>ID, Function</th>
                    <th>Description</th>
                    <th>Involved Subsystems</th>
                </tr>
            </thead>
            <tbody>
            {% for fnc in actor.functions %}
                {% set ports = fnc.inputs + fnc.outputs %}
                {% set outs = (fexs | selectattr("source_port", "in", ports) | list) 
                    | map(attribute="target_port.owner.owner.name") | list %}
                {% set ins = (fexs | selectattr("target_port", "in", ports) | list)
                    | map(attribute="source_port.owner.owner.name") | list %}
                {% set subs = (ins + outs) | unique | list %}
                <tr>
                    <td colspan="1">
                        <p>{{ fnc.uuid }}</p>
                        <p>{{ fnc.name }}</p>
                    </td>
                    <td colspan="1">{{ fnc.description }}</td>
                    <td colspan="1">{{ subs | join(', ') }}</td>
                </tr>
            {% endfor %}
            </tbody>
        </table>
        <p style="text-align: center;"><strong>Functions of {{ actor.name }}</strong></p>
    {% else %}
        <p style="text-align: left;">No actor functions were identified.</p>
    {% endif %}
{% endfor %}
"""